In [13]:
import numpy as np
import rocketcea

In [2]:
# Define Inputs

#Engine Core Properties
A = 0.005771714879 #Chamber Area [m^2]
P_c= 3895540 #Chamber Pressure [Pa]
A_t = 0.0006575790181 #Throat Area [m^2]

g = 9.81 #Gravitational Acceleration [m/s^2]
R = 8.314 #Universal Gas Constant [J/mol*K]
pi = 3.14159265359

#Chamber Gas/Geometric Properties
G_c = 1.2409 #Chamber Gas Specific Heat Ratio
T_c = 3048.54 #Chamber Gas Temperature [K]
M_c = 0.019615 #Chamber Gas Molecular Weight [kg/mol]
Cp_c = 2183.2 #Chamber Gas Specific Heat Capacity [J/kg*K]
mu_c = 0.000092148 #Chamber Gas Viscosity [Pa*s]
k_c = 0.35289 #Chamber Gas Thermal Conductivity [W/m*K]
Pr_c = (Cp_c * mu_c)/(k_c) #Chamber Gas Prandtl Number
R_bar_c = R/M_c #Chamber Gas Specific Gas Constant [J/kg*K]

cstar = ((R_bar_c*T_c/G_c) * ((G_c+1)/2)**((G_c+1)/(G_c-1)))**(0.5) #Characteristic Velocity [m/s]
D_t = 2 * (A_t/pi)**(0.5) #Throat Diameter [m]
r_u = 0.00635 #Throat Radius of Curvature [m]

In [ ]:
# Bartz Iterator Function Definition
# Initial Condition

def Bartz_Iterator(T_c, T_p, Ma, A, h_p, k_w, t_w):
    T_hw = 500 #Hot Wall Temperature Initial Guess [K]
    T_res = 1

    while T_res > 0.001:
        T_old = T_hw
            
        r = Pr_c**0.33 #Recovery Factor
        T_aw = T_c * (((1+(r*(G_c-1)/2)*Ma**2))/((1+((G_c-1)/2)*Ma**2))) #Local Adiabatic Wall Temperature [K]
        s = 1/((((0.5*T_hw/T_c)*(1+((G_c-1)/2)*Ma**2)+0.5)**0.68)*(1+((G_c-1)/2)*Ma**2)**0.12) #Boundary Layer Correction Factor
        h_g = ((0.026/(D_t**0.2))*((mu_c**0.2) * Cp_c/(Pr_c**0.6))*(P_c/cstar)**0.8 * ((D_t/r_u)**0.1))*((A_t/A)**0.9)*s #Bartz Heat Transfer Coefficient [W/m^2*K]
        q = h_g*(T_aw - T_hw)

        T_cw = (q/h_p) + T_p
        T_hw = q*t_w/k_w + T_cw
        T_res = T_hw-T_old
    return(T_hw)

Bartz_Iterator(T_c = T_c, T_p = 330, Ma = 0, A = A, h_p = 17435.70437, k_w = 160, t_w = 0.00381)

789.4597406555492

In [13]:
# Bartz Iteration
# Exit Condition

def Exit_Iterator(T_c, Ma, A, h_p, k_w, t_w):
    T_hwx = 500 #Hot Wall Temperature Initial Guess [K]
    T_px = 330 #Propellant Temperature Initial Guess [K]
    T_res = 1

    while T_res > 0.001:
        T_oldx = T_hwx
            
        r = Pr_c**0.33 #Recovery Factor
        T_aw = T_c * (((1+(r*(G_c-1)/2)*Ma**2))/((1+((G_c-1)/2)*Ma**2))) #Local Adiabatic Wall Temperature [K]
        s = 1/((((0.5*T_hwx/T_c)*(1+((G_c-1)/2)*Ma**2)+0.5)**0.68)*(1+((G_c-1)/2)*Ma**2)**0.12) #Boundary Layer Correction Factor
        h_g = ((0.026/(D_t**0.2))*((mu_c**0.2) * Cp_c/(Pr_c**0.6))*(P_c/cstar)**0.8 * ((D_t/r_u)**0.1))*((A_t/A)**0.9)*s #Bartz Heat Transfer Coefficient [W/m^2*K]
        q = h_g*(T_aw - T_hwx)

        T_cw = (q/h_p) + T_px
        T_hwx = q*t_w/k_w + T_cw
        T_res = T_hwx-T_oldx
    return(T_hwx)

Exit_Iterator(T_c = T_c, Ma = 0, A = A, h_p = 17435.70437, k_w = 160, t_w = 0.00381)

789.4597406555492